In [112]:
import json
from pathlib import Path

In [130]:
path = Path("/home/pau/Documents/datasets/coco2017/annotations")
results = Path("./results/task_b")

For now explore the validation partition, since it is way smaller

In [114]:
with open(path / "captions_train2017.json", 'r') as f_json:
    captions = json.load(f_json)

with open(path / "instances_train2017.json", 'r') as f_json:
    instances = json.load(f_json)

In [115]:
print(list(captions.keys()))
print(list(instances.keys()))

['info', 'licenses', 'images', 'annotations']
['info', 'licenses', 'images', 'annotations', 'categories']


In [116]:
captions["annotations"][:5]

[{'image_id': 203564,
  'id': 37,
  'caption': 'A bicycle replica with a clock as the front wheel.'},
 {'image_id': 322141,
  'id': 49,
  'caption': 'A room with blue walls and a white sink and door.'},
 {'image_id': 16977,
  'id': 89,
  'caption': 'A car that seems to be parked illegally behind a legally parked car'},
 {'image_id': 106140,
  'id': 98,
  'caption': 'A large passenger airplane flying through the air.'},
 {'image_id': 106140,
  'id': 101,
  'caption': 'There is a GOL plane taking off in a partly cloudy sky.'}]

In [117]:
instances["annotations"][:5]

[{'segmentation': [[239.97,
    260.24,
    222.04,
    270.49,
    199.84,
    253.41,
    213.5,
    227.79,
    259.62,
    200.46,
    274.13,
    202.17,
    277.55,
    210.71,
    249.37,
    253.41,
    237.41,
    264.51,
    242.54,
    261.95,
    228.87,
    271.34]],
  'area': 2765.1486500000005,
  'iscrowd': 0,
  'image_id': 558840,
  'bbox': [199.84, 200.46, 77.71, 70.88],
  'category_id': 58,
  'id': 156},
 {'segmentation': [[247.71,
    354.7,
    253.49,
    346.99,
    276.63,
    337.35,
    312.29,
    333.49,
    364.34,
    331.57,
    354.7,
    327.71,
    369.16,
    325.78,
    376.87,
    333.49,
    383.61,
    330.6,
    379.76,
    321.93,
    365.3,
    320.0,
    356.63,
    317.11,
    266.02,
    331.57,
    260.24,
    334.46,
    260.24,
    337.35,
    242.89,
    338.31,
    234.22,
    338.31,
    234.22,
    348.92,
    239.04,
    353.73,
    248.67,
    355.66,
    252.53,
    353.73]],
  'area': 1545.4213000000007,
  'iscrowd': 0,
  'image_id

In [118]:
cat_objs = instances["categories"]

cat_objs.sort(key=lambda x: x["id"])
num_classes = cat_objs[-1]["id"]

In [119]:
class_names = ["" for x in range(num_classes + 1)]

for x in cat_objs:
    class_names[x["id"] - 1] = x["name"]

In [120]:
img_to_class = {}

for x in instances["annotations"]:
    if x["image_id"] not in img_to_class.keys():
        img_to_class[x["image_id"]] = []
    img_to_class[x["image_id"]].append(x["category_id"])

In [121]:
import numpy as np

counts = np.zeros((len(class_names), len(class_names)), dtype=int)

for k, v in img_to_class.items():
    v = list(set(v))
    v.sort()
    for ii in range(len(v)):
        for jj in range(ii, len(v)):
            counts[v[ii] - 1, v[jj] - 1] += 1

In [122]:
n_images = len(instances["images"])

In [123]:
diagonal_ind = np.diag(np.ones(len(class_names), dtype=bool))
class_counts = counts[diagonal_ind]
counts[diagonal_ind] = 0

counts += counts.T

In [124]:
class_probs = class_counts / n_images
jprobs = counts.astype(float) / n_images
cprobs = jprobs / class_probs[:, None]

/tmp/ipykernel_125152/1299758643.py:3: RuntimeWarning: invalid value encountered in true_divide
  cprobs = jprobs / class_probs[:, None]


In [125]:
import pandas as pd

class_probs = pd.DataFrame(class_probs, index=class_names)
counts = pd.DataFrame(counts, index=class_names, columns=class_names)
jprobs = pd.DataFrame(jprobs, index=class_names, columns=class_names)
cprobs = pd.DataFrame(cprobs, index=class_names, columns=class_names)

In [126]:
counts

,person,bicycle,car,motorcycle,airplane,bus,train,truck,boat,traffic light,...,refrigerator,,book,clock,vase,scissors,teddy bear,hair drier,toothbrush,
person,0,2643,8519,2786,1027,3011,1627,3992,1987,2450,...,873,0,2279,1905,839,390,856,98,540,0
bicycle,2643,0,1235,452,17,432,103,448,124,459,...,18,0,71,97,17,5,25,0,4,0
car,8519,1235,0,1377,357,2125,422,3664,262,2534,...,35,0,91,499,40,9,80,2,9,0
motorcycle,2786,452,1377,0,15,293,24,631,22,322,...,5,0,26,51,3,1,15,1,0,0
airplane,1027,17,357,15,0,72,8,644,78,22,...,2,0,8,11,1,0,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
scissors,390,5,9,1,0,0,0,5,0,1,...,31,0,142,35,35,0,9,2,15,0
teddy bear,856,25,80,15,2,8,7,16,11,7,...,12,0,271,46,62,9,0,3,10,0
hair drier,98,0,2,1,1,1,0,0,0,0,...,1,0,11,8,10,2,3,0,21,0
toothbrush,540,4,9,0,0,0,0,1,2,1,...,6,0,52,18,29,15,10,21,0,0


In [127]:
jprobs

,person,bicycle,car,motorcycle,airplane,bus,train,truck,boat,traffic light,...,refrigerator,,book,clock,vase,scissors,teddy bear,hair drier,toothbrush,
person,0.000000,0.022344,0.072020,0.023553,0.008682,0.025455,0.013755,0.033748,0.016798,0.020712,...,0.007380,0.0,0.019267,0.016105,0.007093,0.003297,0.007237,0.000828,0.004565,0.0
bicycle,0.022344,0.000000,0.010441,0.003821,0.000144,0.003652,0.000871,0.003787,0.001048,0.003880,...,0.000152,0.0,0.000600,0.000820,0.000144,0.000042,0.000211,0.000000,0.000034,0.0
car,0.072020,0.010441,0.000000,0.011641,0.003018,0.017965,0.003568,0.030976,0.002215,0.021422,...,0.000296,0.0,0.000769,0.004219,0.000338,0.000076,0.000676,0.000017,0.000076,0.0
motorcycle,0.023553,0.003821,0.011641,0.000000,0.000127,0.002477,0.000203,0.005334,0.000186,0.002722,...,0.000042,0.0,0.000220,0.000431,0.000025,0.000008,0.000127,0.000008,0.000000,0.0
airplane,0.008682,0.000144,0.003018,0.000127,0.000000,0.000609,0.000068,0.005444,0.000659,0.000186,...,0.000017,0.0,0.000068,0.000093,0.000008,0.000000,0.000017,0.000008,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
scissors,0.003297,0.000042,0.000076,0.000008,0.000000,0.000000,0.000000,0.000042,0.000000,0.000008,...,0.000262,0.0,0.001200,0.000296,0.000296,0.000000,0.000076,0.000017,0.000127,0.0
teddy bear,0.007237,0.000211,0.000676,0.000127,0.000017,0.000068,0.000059,0.000135,0.000093,0.000059,...,0.000101,0.0,0.002291,0.000389,0.000524,0.000076,0.000000,0.000025,0.000085,0.0
hair drier,0.000828,0.000000,0.000017,0.000008,0.000008,0.000008,0.000000,0.000000,0.000000,0.000000,...,0.000008,0.0,0.000093,0.000068,0.000085,0.000017,0.000025,0.000000,0.000178,0.0
toothbrush,0.004565,0.000034,0.000076,0.000000,0.000000,0.000000,0.000000,0.000008,0.000017,0.000008,...,0.000051,0.0,0.000440,0.000152,0.000245,0.000127,0.000085,0.000178,0.000000,0.0


The prior is in the rows.

In [128]:
cprobs

,person,bicycle,car,motorcycle,airplane,bus,train,truck,boat,traffic light,...,refrigerator,,book,clock,vase,scissors,teddy bear,hair drier,toothbrush,
person,0.000000,0.041223,0.132871,0.043453,0.016018,0.046962,0.025376,0.062263,0.030991,0.038213,...,0.013616,0.0,0.035546,0.029712,0.013086,0.006083,0.013351,0.001529,0.008422,0.0
bicycle,0.812731,0.000000,0.379766,0.138991,0.005228,0.132841,0.031673,0.137761,0.038130,0.141144,...,0.005535,0.0,0.021833,0.029828,0.005228,0.001538,0.007688,0.000000,0.001230,0.0
car,0.695372,0.100808,0.000000,0.112399,0.029140,0.173455,0.034446,0.299078,0.021386,0.206840,...,0.002857,0.0,0.007428,0.040731,0.003265,0.000735,0.006530,0.000163,0.000735,0.0
motorcycle,0.795545,0.129069,0.393204,0.000000,0.004283,0.083666,0.006853,0.180183,0.006282,0.091947,...,0.001428,0.0,0.007424,0.014563,0.000857,0.000286,0.004283,0.000286,0.000000,0.0
airplane,0.343938,0.005693,0.119558,0.005023,0.000000,0.024113,0.002679,0.215673,0.026122,0.007368,...,0.000670,0.0,0.002679,0.003684,0.000335,0.000000,0.000670,0.000335,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
scissors,0.411827,0.005280,0.009504,0.001056,0.000000,0.000000,0.000000,0.005280,0.000000,0.001056,...,0.032735,0.0,0.149947,0.036959,0.036959,0.000000,0.009504,0.002112,0.015839,0.0
teddy bear,0.400000,0.011682,0.037383,0.007009,0.000935,0.003738,0.003271,0.007477,0.005140,0.003271,...,0.005607,0.0,0.126636,0.021495,0.028972,0.004206,0.000000,0.001402,0.004673,0.0
hair drier,0.518519,0.000000,0.010582,0.005291,0.005291,0.005291,0.000000,0.000000,0.000000,0.000000,...,0.005291,0.0,0.058201,0.042328,0.052910,0.010582,0.015873,0.000000,0.111111,0.0
toothbrush,0.536246,0.003972,0.008937,0.000000,0.000000,0.000000,0.000000,0.000993,0.001986,0.000993,...,0.005958,0.0,0.051639,0.017875,0.028798,0.014896,0.009930,0.020854,0.000000,0.0


In [129]:
class_probs

,0
person,0.542029
bicycle,0.027492
car,0.103570
motorcycle,0.029606
airplane,0.025244
...,...
scissors,0.008006
teddy bear,0.018092
hair drier,0.001598
toothbrush,0.008513


In [131]:
counts.to_csv(results / "counts.csv")
jprobs.to_csv(results / "jprobs.csv")
cprobs.to_csv(results / "cprobs.csv")
class_probs.to_csv(results / "class_probs.csv")

In [132]:
counts.to_markdown(results / "counts.md")
jprobs.to_markdown(results / "jprobs.md")
cprobs.to_markdown(results / "cprobs.md")
class_probs.to_markdown(results / "class_probs.md")